In [1]:
import csv
import json
import time

import boto3
import botocore.session
from boto3.dynamodb.conditions import Key, Attr

In [2]:
file_path = "data"
file_name = 'healthcare-dataset-stroke-data.csv'
table_name="heart_disease"
columns=['id','gender','age','hypertension','heart_disease','ever_married',\
         'work_type','Residence_type','avg_glucose_level','bmi','smoking_status','stroke']

dynamodb = boto3.resource('dynamodb',region_name='us-east-2')

In [3]:
def create_heart_disease_table(table_name):
    try:
        resp = dynamodb.create_table(
            TableName=table_name,
            KeySchema=[
                {
                    'AttributeName': 'id',
                    'KeyType': 'HASH'  # Partition key
                }
            ],
            AttributeDefinitions=[
                {'AttributeName': 'id','AttributeType': 'S'}

            ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 10,
                'WriteCapacityUnits': 10
            }
        )
    except:        
        table = dynamodb.Table(table_name)
        table.delete()
        time.sleep(5)
        create_heart_disease_table(table_name)


In [7]:
def get_json_data(file_path,file_name):
    with open(file_path+"/"+file_name) as f:
        reader = csv.DictReader(f)
        rows = list(reader)        
    return rows

In [8]:
def put_heart_data(table,data):
    with table.batch_writer() as batch:
        with table.batch_writer() as writer:
            for item in data:
                writer.put_item(Item=item)

In [9]:
create_heart_disease_table(table_name)
table = dynamodb.Table(table_name)
heart_disease_data = get_json_data(file_path,file_name)
time.sleep(5) 
put_heart_data(table,heart_disease_data)
